## General Coworker Test

In [1]:
# Testing setup (re-run to clear history)

import asyncio
import pandas as pd
from coworker.modules.coworker import Coworker

coworker = Coworker()
message_history = []

async def print_and_store_results_future(future, task):
    result = await future
    description = task['description']
    
    if description == 'info':
        if result:
            print("Here is some info to begin with:\n")
            print(pd.DataFrame(result))
            print('')
    elif description == 'summary':
        message_history[-1]['content'] += '\n\n' + result
        print(result)
        print('')
    elif description == 'query_and_data':
        query, data = result
        message_history[-1]['content'] += '\n\n' + query
        print(query)
        print('')
        if data != 'Apologies. I cannot figure out a way to answer this request.':
            print(pd.DataFrame(data).head())
        else:
            print(data)

async def main(question):
    message_history.append({"role":"user", "content":question})
    # Create tasks
    tasks = [
        {'task': asyncio.create_task(coworker.get_info_from_text(message_history)), 'description': 'info'},
        {'task': asyncio.create_task(coworker.generate_summary(message_history)), 'description': 'summary'},
        {'task': asyncio.create_task(coworker.process_query_and_data(message_history)), 'description': 'query_and_data'}
    ]
    message_history.append({"role":"assistant", "content":''})

    # Await tasks concurrently and print results as they come
    for task in tasks:
        future = task['task']
        await print_and_store_results_future(future, task)

In [6]:
# Run main coroutine with the question as an argument
question = """Can you aggregate that for the whole year?"""
await main(question)

Here is some info to begin with:

   Code                   Name Exchange CurrencyCode              Sector  \
0  COST  Costco Wholesale Corp   NASDAQ          USD  Consumer Defensive   

          Industry                                        Description  \
0  Discount Stores  Costco Wholesale Corporation, together with it...   

                                                 CEO     IPODate  \
0  {'Name': 'Mr. W. Craig Jelinek', 'Title': 'CEO...  1993-09-22   

         Country      City  
0  United States  Issaquah  

I reviewed Microsoft's quarterly income statements, cash flow statements, balance sheets, and daily share price data for the year . I then calculated the EBITDA for each quarter and summed them up to get the total EBITDA for the year.

SELECT SUM(i.ebitda) AS annual_ebitda
FROM income_statement AS i
JOIN companies AS c ON c.company_id = i.company_id
WHERE c.name ILIKE '%Microsoft%'
AND EXTRACT(YEAR FROM i.date) = 2005;

  Annual Ebitda
0   18404000000


In [7]:
message_history

[{'role': 'user', 'content': "What was apple's ebitda in 2005?"},
 {'role': 'assistant',
  'content': "\n\nI reviewed Apple's annual report for  and extracted the necessary financial data. EBITDA is not directly reported in financial statements, so I calculated it by adding back Interest, Taxes, Depreciation, and Amortization to the Net Income.\n\nSELECT i.ebitda\nFROM income_statement AS i\nJOIN companies AS c ON c.company_id = i.company_id\nWHERE c.name ILIKE '%Apple%'\nAND EXTRACT(YEAR FROM i.date) = 2005;"},
 {'role': 'user',
  'content': 'Can you do the same for Microsoft but give me one number for the whole year?'},
 {'role': 'assistant',
  'content': "\n\nI reviewed Microsoft's annual report for  and extracted the necessary financial data. Similar to Apple, EBITDA is not directly reported in financial statements, so I calculated it by adding back Interest, Taxes, Depreciation, and Amortization to the Net Income. This gave me the EBITDA for the entire year of  for Microsoft.\n\nS

## EOD data test

In [ ]:
from eod import EodHistoricalData
from coworker.config.keys import EOD_KEY
import json
import re

client = EodHistoricalData(EOD_KEY)

with open('coworker/data/companies.json', 'r') as f:
    companies_dict = json.load(f)

In [ ]:
def get_info_from_text(text):
    text = re.sub('[^A-Za-z0-9& ]+', '', text)  # remove special characters
    words_company = [word for word in text.split(' ') if len(word) >= 4]
    words_ticker = [word for word in text.split(' ') if len(word) >= 2]
    
    tickers = []
    for word in words_company:
        for company, ticker in companies_dict.items():
            if word.lower() in company.lower():
                if ticker not in tickers:  # to avoid duplicate entries
                    tickers.append(ticker)
    
    for word in words_ticker:
        for company, ticker in companies_dict.items():
            if word == ticker:
                if ticker not in tickers:  # to avoid duplicate entries
                    tickers.append(ticker)

    info_dict = {
        'Code': [],
        'Name': [],
        'Exchange': [],
        'CurrencyCode': [],
        'Sector': [],
        'Industry': [],
        'Description': [],
        'CEO': [],
        'IPODate': [],
        'Country': [],
        'City': []
    }

    for ticker in tickers:
        info = client.get_fundamental_equity(ticker, filter_='General')
        
        info_dict['Code'].append(info['Code'])
        info_dict['Name'].append(info['Name'])
        info_dict['Exchange'].append(info['Exchange'])
        info_dict['CurrencyCode'].append(info['CurrencyCode'])
        info_dict['Sector'].append(info['Sector'])
        info_dict['Industry'].append(info['Industry'])
        info_dict['Description'].append(info['Description'])
        info_dict['CEO'].append(info['Officers']['0'])
        info_dict['IPODate'].append(info['IPODate'])
        info_dict['Country'].append(info['AddressData']['Country'])
        info_dict['City'].append(info['AddressData']['City'])

    return info_dict

In [ ]:
text = "I would like to see assets and liabilities for Apple, Amazon! and Microsoft and Zoom - Blizzard, ASML, ABT, at&t! ACCENTURE, Costco"
get_info_from_text(text)